In [1]:
from functions.sleeper_functions import *
import pandas as pd
from datetime import datetime

In [2]:
drafts = pd.read_csv('Files/draft_meta.csv')
leagues = pd.read_csv('Files/leagues_info.csv', dtype='object')

In [3]:
leagues = leagues.dropna(subset=['previous_league_id']).drop_duplicates(subset=['league_id']).reset_index(drop=True)

In [4]:
prev = leagues['previous_league_id'].to_list()

In [5]:
el = pd.read_csv('Files/working/league_data_2021.csv', dtype='object')
els = el.league_id.to_list()

print('Original Length: ' + str(len(el)))
start = datetime.now()

prev = leagues['previous_league_id'].to_list()
ll = len(prev)

prev = [i for i in prev if i not in els]

print('Prev removed: ' + str(ll - len(prev)))

conn = http.client.HTTPSConnection('api.sleeper.app')

league_info = pd.DataFrame()
drafts = pd.DataFrame()
slots = pd.DataFrame()
results = pd.DataFrame()
for league in prev[:5000]:
        try:
            conn.request("GET", '/v1/league/' + str(league))
            start_conn = conn.getresponse().read()
            data = json.loads(start_conn)
            
            # League Info
            league_info = league_info.append(pd.json_normalize(data))
            
        except:
            pass
        
        try:
            # Draft Data
            draft_id = data.get('draft_id')
            conn.request("GET", '/v1/draft/' + str(draft_id) )
            start_conn = conn.getresponse().read()
            dataB = json.loads(start_conn)
            if dataB.get('status') != 'complete':
                pass
            else:
                d = pd.json_normalize(dataB)
                sdf = d.filter(like='draft_order')
                d = d[d.columns.drop(list(d.filter(regex='draft_order')))]
                d = d[d.columns.drop(list(d.filter(regex='metadata')))]
                d = d[d.columns.drop(list(d.filter(regex='settings.slots_')))]
                d = d[d.columns.drop(list(d.filter(regex='settings.position_limit_')))]
                drafts = drafts.append(d)

                sdf = sdf.append(pd.DataFrame([sdf.columns], columns = sdf.columns)).reset_index(drop=True)
                sdf.columns = sdf.iloc[0]
                sdf = sdf.loc[1:]
                sdf = sdf.replace('draft_order.', '', regex=True).transpose().reset_index()
                sdf.columns = ['slot','user_id']
                sdf['league_id'] = dataB.get('league_id')
                sdf['status'] = dataB.get('status')
                sdf['draft_id'] = dataB.get('draft_id')
                sdf['draft_type'] = dataB.get('type')
                sdf['season'] = dataB.get('season')
                sdf['scoring_type'] = dataB.get('metadata').get('scoring_type')
                slots = slots.append(sdf)
                
                # 
                try:
                    conn = http.client.HTTPSConnection('api.sleeper.app')
                    conn.request("GET", '/v1/league/' + str(league_id) + '/rosters/')
                    start_conn = conn.getresponse().read()
                    rdata = json.loads(start_conn)
                    df = pd.json_normalize(rdata)
                    df = df[['owner_id','settings.total_moves','settings.wins','settings.ties','settings.losses', 'settings.fpts']]
                    df['wins'] = df['settings.wins'] + (.5 * df['settings.ties'])
                    df['points_standing'] = df['settings.fpts'].rank(ascending=False)
                    df['season_standing'] = df[["wins","settings.fpts"]].apply(tuple,axis=1)\
                                 .rank(method='dense',ascending=False).astype(int)
                    df = df[['owner_id','settings.total_moves','settings.fpts', 'wins', 'points_standing','season_standing']]
                    df.columns = ['user_id','total_moves', 'season_points','wins','points_standing','season_standing']
                    df['league_id'] = league_id
                    results = results.append(df)
                except:
                    pass
            
        except:
            pass

slots = slots.merge(results, on=['league_id','user_id'], how='left').drop_duplicates()     

league_info['last_update_dt'] = datetime.now()
drafts['last_update_dt'] = datetime.now()
slots['last_update_dt'] = datetime.now()

# Handle League Info:

el = el.append(league_info).sort_values(by=['last_update_dt','last_message_time'],ascending=[False,False]).drop_duplicates(subset=['league_id', 'draft_id'])
print('New Length: ' + str(len(el)))

el.to_csv('Files/working/league_data_2021.csv', index=False)
del el

# Handle Draft Info:

ed = pd.read_csv('Files/working/draft_data_2021.csv', dtype='object')
ed = ed.append(drafts).sort_values(by=['last_update_dt','last_message_time'],ascending=[False,False]).drop_duplicates(subset=['league_id','draft_id'])
ed.to_csv('Files/working/draft_data_2021.csv', index=False)

es = pd.read_csv('Files/working/slots_2021.csv', dtype='object')
es = es.append(slots).drop_duplicates(subset=['league_id', 'draft_id','user_id'])
es.to_csv('Files/working/slots_2021.csv', index=False)

#league_info.drop_duplicates(subset=['league_id','draft_id']).to_csv('Files/working/league_data_2021.csv', index=False)
#drafts.drop_duplicates(subset=['league_id','draft_id']).to_csv('Files/working/draft_data_2021.csv', index=False)
#slots.drop_duplicates(subset=['league_id', 'draft_id','user_id']).to_csv('Files/working/slots_2021.csv', index=False)





print('Finished in: ' + str(datetime.now()-start))

Original Length: 8096
Prev removed: 8183


/var/folders/w2/0q5xhrf578d2c6cdn9g187jm0000gn/T/ipykernel_61298/1033146880.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  league_info['last_update_dt'] = datetime.now()


New Length: 12802
Finished in: 0:22:00.821659
